In [23]:
# import argparse
from glob import glob
import cv2
import pandas as pd
import os
import subprocess
# import the necessary packages
import screeninfo
#Global Var
screen_id = 0
is_color = True
CLASSES = ["dark_corners", "hair", "gel_border", "gel_bubble", "rulers", "ink", "patches", "vasc"]
# parser = argparse.ArgumentParser(prog='ImgAnn.py', description="This Program is designed to annotated image for retrieval problems", formatter_class=argparse.MetavarTypeHelpFormatter)
# parser.add_argument("--input_path","-i", required=True, type=str ,nargs='+',
#                     help="Path to images directory.\
#                     \n You can also input the name of each image.")
# parser.add_argument("--input_csv", "-csv", type=str ,nargs=1, default='', metavar=("csv"),
#                     help="Load the csv already created with this program")                    
# args = vars(parser.parse_args())
# get the size of the screen

#Functions
def create_relevance_df(img_list,csv_path=""):
    if csv_path:
        return pd.read_csv(csv_path)
    filename = img_list.copy()
    _class = [[0 for i in CLASSES]  for img in img_list]
    data = {"Filename":filename, "Class" : _class}
    df = pd.DataFrame.from_dict(data)
    return df
def save_df(df):
    df.to_csv("img_ann.csv",index = False)
    
def load_df():
    #olnames=['Filename', 'Relevance']
    return pd.read_csv("img_ann.csv",index_col = False)

def image_annotation(image_list,load_csv=""):
    screen = screeninfo.get_monitors()[screen_id]
    width, height = screen.width, screen.height
    index_image = 0
    window_name = 'Class Annotation'
    cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.moveWindow(window_name, screen.x - 1, screen.y - 1)
    cv2.setWindowProperty(window_name, int(cv2.WND_PROP_FULLSCREEN/2),int(cv2.WINDOW_FULLSCREEN/2))
    df_classes = create_relevance_df(image_list,load_csv)
    global refPt
    global cont 
    cont = 1
    refPt = [None,None]
    
    while(1):
        imagePath  = df_classes.loc[index_image]['Filename']
        csv_classes = df_classes.loc[index_image]['Class']
        image = cv2.imread(imagePath)
        if type(csv_classes) == str:
            csv_classes = eval(csv_classes)
        img_class = csv_classes
        text = "Class - "
        for index, cl in enumerate(csv_classes):
            if cl == 1:
                text += CLASSES[index] + ", "                                                                                                                                                                                                
        color = (20,200,20)      
        cv2.putText(image, "{} {}/{} ".format(text, index_image+1,len(image_list)), (10, 30),                                                                                                                                             
        cv2.FONT_HERSHEY_TRIPLEX, 0.5, color, 1)                                                                                                                                                                                      
        cv2.imshow(window_name, image)
        key = cv2.waitKey(5)
        
        if key == ord('s'):
            save_df(df_classes)
            text = "SAVED"                                                                                                                                                                                              
            color = (0,0,255)      
            cv2.putText(image, "{} {}/{} ".format(text, index_image+1,len(image_list)), (10, 30),                                                                                                                                             
                cv2.FONT_HERSHEY_DUPLEX, 0.8, color, 2)                                                                                                                                                                                      
            cv2.imshow(window_name, image)
            key = cv2.waitKey(0)
            continue
            
        # Keymap for each class
        keys_cls = [ord('y'),ord('u'),ord('i'),ord('o'),
                   ord('h'),ord('j'),ord('k'),ord('l')]
        # Change label
        if key in keys_cls :
            csv_classes[keys_cls.index(key)] = int(not bool(csv_classes[keys_cls.index(key)]))
            df_classes.loc[index_image]["Class"] = csv_classes
            save_df(df_classes)
            continue
            
        # Keymap for forward and backward
        if key==ord('d'):  # normally -1 returned,so don't print it
            index_image +=1
            index_image = index_image % len(image_list)
            save_df(df_classes)
            
        elif key==ord('a'):
            index_image -= 1
            index_image = index_image % len(image_list)
            save_df(df_classes)
            
        elif key==27:
            print("ESCAPE")
            break # else print its value
    cv2.destroyAllWindows()

In [24]:
path_name = <INSERT_PATH_NAME>
img_list = glob(path_name)
# Check if csv exists
csv = "img_ann.csv"
if not os.path.exists(csv):
    csv_df = create_relevance_df(img_list)
    save_df(csv_df)
image_annotation(img_list,csv)

ESCAPE
